In [1]:
from pyspark.sql import SparkSession
import pyspark.sql.functions as F
from pyspark.sql import Window
import matplotlib.pyplot as plt
from pyspark import StorageLevel

In [2]:
spark = SparkSession.builder \
    .appName('AV-parking-area-analysis') \
    .config("spark.sql.adaptive.enabled", "true") \
    .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
    .config("spark.memory.fraction", "0.8") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.sql.shuffle.partitions", "400") \
    .getOrCreate()

26/02/11 08:58:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### 음식점 (처인구)

In [3]:
store_df_gg = (spark.read
      .option('header', True)
      .option('inferSchema', True)
      .csv('../data/store/소상공인시장진흥공단_상가(상권)정보_경기_202512.csv'))

store_df = store_df_gg.filter(
    (F.col("시군구명") == "용인시 처인구") &
    (F.col("상권업종대분류코드") == "I2") 
)

In [4]:
store_df = store_df.withColumnRenamed("지번코드", "고유번호")

In [5]:
store_df.count()

4372

In [6]:
store_unique_keys = (store_df
                    .groupBy("고유번호")
                    .count()
                    .filter(F.col("count") == 1)
                    .select("고유번호"))

store_df = store_df.join(store_unique_keys, "고유번호", "inner")
store_df.count()

1905

### 토지소유정보(처인구)

In [7]:
ground_own_path = "../data/ground/소유정보/경기도_토지소유정보.csv" 

ground_df = spark.read \
    .option("header", "true") \
    .option("encoding", "UTF-8") \
    .csv(ground_own_path)

In [8]:
ground_df = ground_df.filter(F.col("법정동코드").startswith("41461"))

In [9]:
ground_df = ground_df.select(
    "고유번호", 
    "법정동명", 
    "지번", 
    "소유구분코드", 
    "소유구분",
    "소유권변동원인코드",
    "소유권변동원인",
    "소유권변동일자",
    "공유인수",
    "토지면적",
    "지목코드",
    "지목",
    "공시지가",
    "기준연월"
)


In [10]:
# 1. 소유 정보에서 필지당 1명만 추출 (최신 소유자)
window_spec = Window.partitionBy("고유번호").orderBy(F.col("소유권변동일자").desc())

filtered_ground_df = ground_df \
    .withColumn("row_num", F.row_number().over(window_spec)) \
    .filter(F.col("row_num") == 1) \
    .drop("row_num")

filtered_ground_df.count()

267670

In [11]:
filtered_ground_df = filtered_ground_df.withColumn("split_jibun", F.split(F.col("지번"), "-")) \
    .withColumn("주번", F.get(F.col("split_jibun"), 0)) \
    .withColumn("부번", F.get(F.col("split_jibun"), 1)) \
    .drop("split_jibun") # 중간 계산용 컬럼 삭제

### 건축물

In [12]:
building_path = "../data/building/mart_djy_03.txt"


building_columns = [
    "관리_건축물대장_PK", "대장_구분_코드", "대장_구분_코드_명", "대장_종류_코드", 
    "대장_종류_코드_명", "대지_위치", "도로명_대지_위치", "건물_명", "시군구_코드", 
    "법정동_코드", "대지_구분_코드", "번", "지", "특수지_명", "블록", "로트", 
    "외필지_수", "새주소_도로_코드", "새주소_법정동_코드", "새주소_지상지하_코드", 
    "새주소_본_번", "새주소_부_번", "동_명", "주_부속_구분_코드", "주_부속_구분_코드_명", 
    "대지_면적(㎡)", "건축_면적(㎡)", "건폐_율(%)", "연면적(㎡)", "용적_률_산정_연면적(㎡)", 
    "용적_률(%)", "구조_코드", "구조_코드_명", "기타_구조", "주_용도_코드", 
    "주_용도_코드_명", "기타_용도", "지붕_코드", "지붕_코드_명", "기타_지붕", 
    "세대_수(세대)", "가구_수(가구)", "높이(m)", "지상_층_수", "지하_층_수", 
    "승용_승강기_수", "비상용_승강기_수", "부속_건축물_수", "부속_건축물_면적(㎡)", 
    "총_동_연면적(㎡)", "옥내_기계식_대수(대)", "옥내_기계식_면적(㎡)", "옥외_기계식_대수(대)", 
    "옥외_기계식_면적(㎡)", "옥내_자주식_대수(대)", "옥내_자주식_면적(㎡)", "옥외_자주식_대수(대)", 
    "옥외_자주식_면적(㎡)", "허가_일", "착공_일", "사용승인_일", "허가번호_년", 
    "허가번호_기관_코드", "허가번호_기관_코드_명", "허가번호_구분_코드", "허가번호_구분_코드_명", 
    "호_수(호)", "에너지효율_등급", "에너지절감_율", "에너지_EPI점수", "친환경_건축물_등급", 
    "친환경_건축물_인증점수", "지능형_건축물_등급", "지능형_건축물_인증점수", "생성_일자", 
    "내진_설계_적용_여부", "내진_능력"
]

building_df = (spark.read
               .option("header", False)
               .option("sep", "|")
               .option("encoding", "UTF-8")
               .csv(building_path))

building_df = building_df.toDF(*building_columns)

In [13]:
building_df = building_df.filter(F.col("시군구_코드") == ("41461"))

In [14]:
# 건축물대장(building_df)의 대지구분코드를 토지정보 PNU 표준(1, 2)으로 변환
building_df = building_df.withColumn("pnu_land_gb", 
    F.when(F.col("대지_구분_코드") == "0", "1")  # 대지(0) -> PNU용(1)
     .when(F.col("대지_구분_코드") == "1", "2")  # 산(1) -> PNU용(2)
     .otherwise(F.col("대지_구분_코드"))         
)

building_df = building_df.withColumn(
    "고유번호",
    F.concat(
        F.col("시군구_코드"),          # 5
        F.col("법정동_코드"),          # 5
        F.col("pnu_land_gb"),       
        F.lpad(F.col("번"), 4, "0"),     # 본번
        F.lpad(F.col("지"), 4, "0")      # 부번
    )
)

In [15]:
building_df.count()

51701

In [16]:
building_unique_keys = (store_df
                    .groupBy("고유번호")
                    .count()
                    .filter(F.col("count") == 1)
                    .select("고유번호"))

building_df = building_df.join(store_unique_keys, "고유번호", "inner")
building_df.count()

2310

### join

In [26]:
land = (filtered_ground_df
        .select("고유번호", "법정동명", "주번", "지번", "소유권변동일자", "지목코드", "지목", "토지면적")
        .withColumn("토지면적", F.col("토지면적").cast("double"))
       )

target_jimok = ["08", "09", "11", "13", "28"] 
land = land.filter(
    (F.col("지목코드").isin(target_jimok))
)

store = (store_df
        .select("고유번호", "상호명", "지점명", "지번주소", "경도", "위도")
       )

building = (building_df
        .select("고유번호", "옥외_자주식_면적(㎡)")
        .withColumn("옥외_자주식_면적(㎡)", F.col("옥외_자주식_면적(㎡)").cast("double"))
       )



In [27]:
land_building = land.join(building, on="고유번호", how="left")

In [28]:
land_building.count()

60221

In [29]:
all_joined = land_building.join(store, on="고유번호", how="left")

In [30]:
all_joined.count()

60221

In [35]:
target_dong = "경기도 용인시 처인구 삼가동"
target_jubun = "250"
target_date = "2021-02-08"

all_joined.filter(
    (F.col("법정동명") == target_dong) & 
    (F.col("주번") == target_jubun) & 
    (F.col("소유권변동일자") == target_date)
).show(truncate=False)

+-------------------+---------------------------+----+------+--------------+--------+----+--------+--------------------+------------+------+---------------------------------+----------------+----------------+
|고유번호           |법정동명                   |주번|지번  |소유권변동일자|지목코드|지목|토지면적|옥외_자주식_면적(㎡)|상호명      |지점명|지번주소                         |경도            |위도            |
+-------------------+---------------------------+----+------+--------------+--------+----+--------+--------------------+------------+------+---------------------------------+----------------+----------------+
|4146110300102500083|경기도 용인시 처인구 삼가동|250 |250-83|2021-02-08    |08      |대  |444.0   |NULL                |NULL        |NULL  |NULL                             |NULL            |NULL            |
|4146110300102500084|경기도 용인시 처인구 삼가동|250 |250-84|2021-02-08    |08      |대  |444.0   |NULL                |NULL        |NULL  |NULL                             |NULL            |NULL            |
|4146110300102500003|경기도 용인시 처인구

In [31]:
result_df = all_joined.groupBy("법정동명", "주번", "소유권변동일자") \
    .agg(
        # '상호명'이 null이 아닌 것들을 모아서 쉼표(,)로 연결 (음식점 목록)
        F.concat_ws(",", F.collect_set("상호명")).alias("음식점 목록"),
        
        # 모든 행의 '지번' 을 모아서 쉼표(,)로 연결 (부번 목록)
        F.concat_ws(",", F.collect_set("지번")).alias("지번 목록"),
        
        F.sum(
            F.when(
                (F.col("상호명").isNull()) & (F.col("옥외_자주식_면적(㎡)").isNull()), 
                F.col("토지면적")
            )
            .when((F.col("상호명").isNotNull()), F.col("옥외_자주식_면적(㎡)"))
            .otherwise(0),
        ).alias("면적"),
        
        F.count("상호명").alias("biz_count")
    ) \
    .filter(F.col("biz_count") > 0) \
    .drop("biz_count")


In [32]:
result_df = result_df.filter(
    (F.col("면적") >= 300)
)

In [33]:
result_df.count()

187

In [34]:
output_path = "../data/output/용인시_결과"

result_df.coalesce(1).write \
    .mode("overwrite") \
    .option("header", "true") \
    .option("encoding", "UTF-8") \
    .csv(output_path)